# Elevator Acceleration Experiment

On Thursday 2016-10-20, I tracked the acceleration in a Dubai Mall elevator by placing a Samsung Galaxy S6 flat on the floor and using the onboard accelerometer, with the help of the SparkVue app, logged the acceleration in the Z axis data for 12 seconds at 500 Hz. The data is stored in csv format, and I have sliced off the final few seconds where I was picking up my phone and discarded all the irrelevant columns, keeping only time and acceleration in z axis.

## Libraries
I will be using pandas and plotly to analyze this data on python 3.5 (matplotlib graphs exist but have been commented out).

In [1]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import plotly.plotly as py
import plotly.graph_objs as go

%matplotlib inline

## Importing data
The data will be loaded from the csv file. The first five rows of a ~6000 row table:

In [2]:
df = pd.read_csv('elevator.csv')
df.head()

,time,acceleration
0,0.000,9.8
1,0.002,9.8
2,0.004,9.8
3,0.006,9.8
4,0.008,9.8


## Raw acceleration against time
The raw vertical acceleration measured by the phone can be plotted against time straight away.

In [3]:
#accel_time = df.set_index('time')
#accel_plot = accel_time.plot(kind = 'line', legend = None, title = 'Acceleration against time in elevator')
#accel_plot.set_xlabel('Time (s)')
#accel_plot.set_ylabel('Acceleration (ms^-2)')

acc_time = [
    go.Scatter(
        x=df['time'], # assign x as the dataframe column 'x'
        y=df['acceleration']
    )
]

acc_time_layout = go.Layout(
    title='acceleration against time',
    yaxis=dict(title='acceleration (ms^-2)'),
    xaxis=dict(title='time (s)')
)

acc_time_fig = go.Figure(data = acc_time, layout = acc_time_layout)
py.iplot(acc_time_fig)

With this plot, a limitation is shown of the data, namely that only .1 ms^-2 of accuracy has been measured to. Also, it is demonstrated that the resting acceleration of ~ 9.8 ms^-2 must be subtracted from the data.

## Subtracting relative acceleration
The mean of the raw acceleration in the first 2.5 seconds of the data is taken and subtracted from all of the acceleration values to give relative acceleration which can then be plotted against time.

In [4]:
resting_accel = df.loc[df['time'] <= 2.5]['acceleration'].mean()
df['relative acceleration'] = df.apply(lambda row: row['acceleration'] - resting_accel, axis = 1)
df.head()

,time,acceleration,relative acceleration
0,0.000,9.8,-0.009912
1,0.002,9.8,-0.009912
2,0.004,9.8,-0.009912
3,0.006,9.8,-0.009912
4,0.008,9.8,-0.009912


In [5]:
#rel_accel_time = df[['time','relative acceleration']].set_index('time')
#rel_accel_plot = rel_accel_time.plot(kind = 'line', legend = None, title = 'Relative acceleration against time in elevator')
#rel_accel_plot.set_xlabel('Time (s)')
#rel_accel_plot.set_ylabel('Acceleration (ms^-2)')

rel_acc_time = [
    go.Scatter(
        x=df['time'], # assign x as the dataframe column 'x'
        y=df['relative acceleration']
    )
]

rel_acc_time_layout = go.Layout(
    title='relative acceleration against time',
    yaxis=dict(title='acceleration (ms^-2)'),
    xaxis=dict(title='time (s)')
)

rel_acc_time_fig = go.Figure(data = rel_acc_time, layout = rel_acc_time_layout)
py.iplot(rel_acc_time_fig)

In [9]:
max_accel = df.query('4.5 <= time <= 6.55')['relative acceleration'].mean()
max_decel = df.query('8.9 <= time <= 10.95')['relative acceleration'].mean()
print('max accel:', max_accel)
print('max decel:', max_decel)

max accel: 0.606657130436
max decel: -0.596169380285


Now we can see that the elevator acceleration reaches a maximum of ~ 0.61 ms^-2, and a maximum deceleration of ~ -0.60 ms^-2.

## Velocity
Finally, we can approximate the velocity by taking the cumulative sum of the relative acceleration multiplied by the time gap between readings, and then plot this.

In [10]:
df['velocity'] = (df['time'].diff().fillna(0) * df['relative acceleration']).cumsum()
df.head()

,time,acceleration,relative acceleration,velocity
0,0.000,9.8,-0.009912,-0.000000
1,0.002,9.8,-0.009912,-0.000020
2,0.004,9.8,-0.009912,-0.000040
3,0.006,9.8,-0.009912,-0.000059
4,0.008,9.8,-0.009912,-0.000079


In [11]:
#vel_time = df[['time','velocity']].set_index('time')
#vel_plot = vel_time.plot(kind = 'line', legend = None, title = 'Velocity against time in elevator')
#vel_plot.set_xlabel('Time (s)')
#vel_plot.set_ylabel('Velocity (ms^-1)')

vel_time = [
    go.Scatter(
        x=df['time'], # assign x as the dataframe column 'x'
        y=df['velocity']
    )
]

vel_time_layout = go.Layout(
    title='relative acceleration against time',
    yaxis=dict(title='velocity (ms^-1)'),
    xaxis=dict(title='time (s)')
)

vel_time_fig = go.Figure(data = vel_time, layout = vel_time_layout)
py.iplot(vel_time_fig)

In [12]:
print('max velocity:', df['velocity'].max())

max velocity: 1.63212853717


The velocity seems to peak at 1.63 ms^-1 in this elevator. However, as demonstrated by the fact that the velocity doesn't reach 0 at the end of the journey, this dataset is not very accurate, mostly due to the poor accuracy of the Galaxy S6's onboard accelerometer. This is very likely the reason for the failure to calculate an accurate velocity for the journey.

Regardless, this has been an interesting exercise in datalogging and analysis of data using Python.